In [7]:
from datetime import datetime
import pandas as pd
import pyodbc
import requests
import os


# Static Declarations.
server = '10.10.69.97'
database = 'NTPCPowerBI'
username = 'sa'
password = 'Ntpc@123'

#https://srldc.in/var/ftp/schedule/WBES_XML/FullSchedule-09-11-2021.xml
#https://srldc.in/WBES_XML
rldc_url = 'https://srldc.in/var/ftp/schedule/WBES_XML/'
url="https://srldc.in/WBES_XML"

# DataBase Connc

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server +
                      ';DATABASE='+database+';UID='+username+';PWD=' + password)

cursor = cnxn.cursor()

In [8]:
# Get Data
max_time = cursor.execute("select max(FILETIME) from NTPCPowerBI.dbo.SRLDC_XML_PortingStatus with (nolock)")
max_time = cursor.fetchall()
data = [list(rows) for rows in max_time]
print(max_time)
maxdbtime = data[0]
maxdbtime = pd.to_datetime(maxdbtime)
print(maxdbtime)

[(datetime.datetime(2021, 11, 9, 18, 24, 58), )]
DatetimeIndex(['2021-11-09 18:24:58'], dtype='datetime64[ns]', freq=None)


In [9]:
def dataparser(url, dbtime):

    rawtable = pd.read_html(url)

    df = pd.DataFrame(rawtable[0])
    df.drop('Select',axis=1,inplace=True)
    

    #df["Last Write Time"] = pd.to_datetime(df.Last Write Time, format="%m/%d/%Y %I:%M:%S %p")
    df['Last Write Time'] = df['Last Write Time'].apply(lambda x: pd.Timestamp(x).strftime('%d-%m-%Y %H:%M:%S %p'))


    

    df['srldc'] = 'SRLDC'

    df['porting_status'] = 'N'
    gf=df.rename(columns={'File Name': 'FileName', 'Last Write Time': 'Time','File Size':'FileSize'})

    #finadf = df[df["Last Write Time"] > dbtime]

    #return finadf
    return gf


finaldata = dataparser(url, maxdbtime[0])

finaldata.head(5)

,FileName,Time,FileSize,srldc,porting_status
0,FullSchedule-11-11-2021.xml,11-11-2021 15:11:26 PM,5487 kb,SRLDC,N
1,FullSchedule-(137)-11-11-2021.xml,11-11-2021 15:11:26 PM,5487 kb,SRLDC,N
2,FullSchedule-(136)-11-11-2021.xml,11-11-2021 15:05:46 PM,5487 kb,SRLDC,N
3,FullSchedule-(135)-11-11-2021.xml,11-11-2021 14:54:51 PM,5486 kb,SRLDC,N
4,FullSchedule-(134)-11-11-2021.xml,11-11-2021 14:51:47 PM,5489 kb,SRLDC,N


In [10]:
if len(finaldata) < 1:
    print("No New  Records")
else:
    print(f'Total New Records : {len(finaldata)}')

Total New Records : 1578


In [11]:
finaldata

,FileName,Time,FileSize,srldc,porting_status
0,FullSchedule-11-11-2021.xml,11-11-2021 15:11:26 PM,5487 kb,SRLDC,N
1,FullSchedule-(137)-11-11-2021.xml,11-11-2021 15:11:26 PM,5487 kb,SRLDC,N
2,FullSchedule-(136)-11-11-2021.xml,11-11-2021 15:05:46 PM,5487 kb,SRLDC,N
3,FullSchedule-(135)-11-11-2021.xml,11-11-2021 14:54:51 PM,5486 kb,SRLDC,N
4,FullSchedule-(134)-11-11-2021.xml,11-11-2021 14:51:47 PM,5489 kb,SRLDC,N
...,...,...,...,...,...
1573,FullSchedule-(19)-04-11-2021.xml,11-04-2021 00:35:55 AM,5299 kb,SRLDC,N
1574,FullSchedule-(18)-04-11-2021.xml,11-04-2021 00:25:01 AM,5298 kb,SRLDC,N
1575,FullSchedule-(17)-04-11-2021.xml,11-04-2021 00:20:57 AM,5298 kb,SRLDC,N
1576,FullSchedule-(16)-04-11-2021.xml,11-04-2021 00:09:44 AM,5290 kb,SRLDC,N


In [12]:
for index, row in finaldata.iterrows():
    print("Downloading Record "+str(row.FileName))
    # print(str(row.Filename))
    url_new="https://srldc.in/var/ftp/schedule/WBES_XML/"
    #urllib.request.urlopen(url_new).read().decode('utf-8')
    download_url = url_new + row.FileName
    try:
        response = requests.get(download_url,timeout=None)
    except:
        print("error server")

    folderpath = r"C:\NTPC\srldc\srldc"
    filename = os.path.join(folderpath, row.FileName)

    # print(response.content)

    with open(filename, 'wb') as file:
        file.write(response.content)
    print("Done")

error server


NameError: name 'response' is not defined

In [54]:
# Part 3 - Insert Data into DB

insertquery = "INSERT INTO NTPCPowerBI.dbo.SRLDC_XML_PortingStatus (FileName,FileTime,FileSize,SRLDC,Porting_Status) VALUES (?,?,?,?,'N')"
for index, row in finaldata.iterrows():
    print("Inserting Record "+str(row.FileName))
    cursor.execute(insertquery, row.FileName, row.Time,row.FileSize,row.srldc)
    #print("Downloading Record "+str(row.Filename))
    # print(str(row.Filename))
    srldc_url="https://srldc.in/var/ftp/schedule/WBES_XML/"
    download_url = srldc_url + row.FileName

    #response = requests.get(download_url)

    folderpath = r"C:\NTPC\srldc\srldc"
    filename = os.path.join(folderpath, row.FileName)

    # print(response.content)

    #with open(filename, 'wb') as file:
        #file.write(response.content)
    #print("Done")
    cnxn.commit()
    # cursor.clos

Inserting Record FullSchedule-09-11-2021.xml
Inserting Record FullSchedule-(152)-09-11-2021.xml
Inserting Record FullSchedule-(151)-09-11-2021.xml
Inserting Record FullSchedule-(150)-09-11-2021.xml
Inserting Record FullSchedule-10-11-2021.xml
Inserting Record FullSchedule-(0)-10-11-2021.xml
Inserting Record FullSchedule-(149)-09-11-2021.xml
Inserting Record FullSchedule-(148)-09-11-2021.xml
Inserting Record FullSchedule-(147)-09-11-2021.xml
Inserting Record FullSchedule-(146)-09-11-2021.xml
Inserting Record FullSchedule-(145)-09-11-2021.xml
Inserting Record FullSchedule-(144)-09-11-2021.xml
Inserting Record FullSchedule-(143)-09-11-2021.xml
Inserting Record FullSchedule-(142)-09-11-2021.xml
Inserting Record FullSchedule-(141)-09-11-2021.xml
Inserting Record FullSchedule-(140)-09-11-2021.xml
Inserting Record FullSchedule-(139)-09-11-2021.xml
Inserting Record FullSchedule-(138)-09-11-2021.xml
Inserting Record FullSchedule-(137)-09-11-2021.xml
Inserting Record FullSchedule-(136)-09-11-202

Inserting Record FullSchedule-(179)-08-11-2021.xml
Inserting Record FullSchedule-(1)-09-11-2021.xml
Inserting Record FullSchedule-(178)-08-11-2021.xml
Inserting Record FullSchedule-(177)-08-11-2021.xml
Inserting Record FullSchedule-(176)-08-11-2021.xml
Inserting Record FullSchedule-(175)-08-11-2021.xml
Inserting Record FullSchedule-(174)-08-11-2021.xml
Inserting Record FullSchedule-(173)-08-11-2021.xml
Inserting Record FullSchedule-(172)-08-11-2021.xml
Inserting Record FullSchedule-(171)-08-11-2021.xml
Inserting Record FullSchedule-(170)-08-11-2021.xml
Inserting Record FullSchedule-(169)-08-11-2021.xml
Inserting Record FullSchedule-(168)-08-11-2021.xml
Inserting Record FullSchedule-(167)-08-11-2021.xml
Inserting Record FullSchedule-(166)-08-11-2021.xml
Inserting Record FullSchedule-(165)-08-11-2021.xml
Inserting Record FullSchedule-(164)-08-11-2021.xml
Inserting Record FullSchedule-(163)-08-11-2021.xml
Inserting Record FullSchedule-(162)-08-11-2021.xml
Inserting Record FullSchedule-(16

Inserting Record FullSchedule-(10)-08-11-2021.xml
Inserting Record FullSchedule-(205)-07-11-2021.xml
Inserting Record FullSchedule-(9)-08-11-2021.xml
Inserting Record FullSchedule-(204)-07-11-2021.xml
Inserting Record FullSchedule-(8)-08-11-2021.xml
Inserting Record FullSchedule-(203)-07-11-2021.xml
Inserting Record FullSchedule-(7)-08-11-2021.xml
Inserting Record FullSchedule-(202)-07-11-2021.xml
Inserting Record FullSchedule-(6)-08-11-2021.xml
Inserting Record FullSchedule-(5)-08-11-2021.xml
Inserting Record FullSchedule-(201)-07-11-2021.xml
Inserting Record FullSchedule-(200)-07-11-2021.xml
Inserting Record FullSchedule-(4)-08-11-2021.xml
Inserting Record FullSchedule-(198)-07-11-2021.xml
Inserting Record FullSchedule-(197)-07-11-2021.xml
Inserting Record FullSchedule-(3)-08-11-2021.xml
Inserting Record FullSchedule-(196)-07-11-2021.xml
Inserting Record FullSchedule-(195)-07-11-2021.xml
Inserting Record FullSchedule-(194)-07-11-2021.xml
Inserting Record FullSchedule-(193)-07-11-2021

Inserting Record FullSchedule-(49)-07-11-2021.xml
Inserting Record FullSchedule-(48)-07-11-2021.xml
Inserting Record FullSchedule-(47)-07-11-2021.xml
Inserting Record FullSchedule-(46)-07-11-2021.xml
Inserting Record FullSchedule-(45)-07-11-2021.xml
Inserting Record FullSchedule-(44)-07-11-2021.xml
Inserting Record FullSchedule-(43)-07-11-2021.xml
Inserting Record FullSchedule-(42)-07-11-2021.xml
Inserting Record FullSchedule-(41)-07-11-2021.xml
Inserting Record FullSchedule-(40)-07-11-2021.xml
Inserting Record FullSchedule-(39)-07-11-2021.xml
Inserting Record FullSchedule-(38)-07-11-2021.xml
Inserting Record FullSchedule-(37)-07-11-2021.xml
Inserting Record FullSchedule-(36)-07-11-2021.xml
Inserting Record FullSchedule-(35)-07-11-2021.xml
Inserting Record FullSchedule-(34)-07-11-2021.xml
Inserting Record FullSchedule-(33)-07-11-2021.xml
Inserting Record FullSchedule-(32)-07-11-2021.xml
Inserting Record FullSchedule-(31)-07-11-2021.xml
Inserting Record FullSchedule-(30)-07-11-2021.xml


Inserting Record FullSchedule-(93)-06-11-2021.xml
Inserting Record FullSchedule-(92)-06-11-2021.xml
Inserting Record FullSchedule-(91)-06-11-2021.xml
Inserting Record FullSchedule-(90)-06-11-2021.xml
Inserting Record FullSchedule-(89)-06-11-2021.xml
Inserting Record FullSchedule-(88)-06-11-2021.xml
Inserting Record FullSchedule-(87)-06-11-2021.xml
Inserting Record FullSchedule-(86)-06-11-2021.xml
Inserting Record FullSchedule-(85)-06-11-2021.xml
Inserting Record FullSchedule-(84)-06-11-2021.xml
Inserting Record FullSchedule-(83)-06-11-2021.xml
Inserting Record FullSchedule-(82)-06-11-2021.xml
Inserting Record FullSchedule-(81)-06-11-2021.xml
Inserting Record FullSchedule-(80)-06-11-2021.xml
Inserting Record FullSchedule-(79)-06-11-2021.xml
Inserting Record FullSchedule-(78)-06-11-2021.xml
Inserting Record FullSchedule-(77)-06-11-2021.xml
Inserting Record FullSchedule-(76)-06-11-2021.xml
Inserting Record FullSchedule-(75)-06-11-2021.xml
Inserting Record FullSchedule-(74)-06-11-2021.xml


Inserting Record FullSchedule-(139)-05-11-2021.xml
Inserting Record FullSchedule-(138)-05-11-2021.xml
Inserting Record FullSchedule-(137)-05-11-2021.xml
Inserting Record FullSchedule-(136)-05-11-2021.xml
Inserting Record FullSchedule-(135)-05-11-2021.xml
Inserting Record FullSchedule-(134)-05-11-2021.xml
Inserting Record FullSchedule-(133)-05-11-2021.xml
Inserting Record FullSchedule-(132)-05-11-2021.xml
Inserting Record FullSchedule-(131)-05-11-2021.xml
Inserting Record FullSchedule-(130)-05-11-2021.xml
Inserting Record FullSchedule-(129)-05-11-2021.xml
Inserting Record FullSchedule-(128)-05-11-2021.xml
Inserting Record FullSchedule-(127)-05-11-2021.xml
Inserting Record FullSchedule-(126)-05-11-2021.xml
Inserting Record FullSchedule-(125)-05-11-2021.xml
Inserting Record FullSchedule-(124)-05-11-2021.xml
Inserting Record FullSchedule-(123)-05-11-2021.xml
Inserting Record FullSchedule-(122)-05-11-2021.xml
Inserting Record FullSchedule-(121)-05-11-2021.xml
Inserting Record FullSchedule-(

Inserting Record FullSchedule-(174)-04-11-2021.xml
Inserting Record FullSchedule-(173)-04-11-2021.xml
Inserting Record FullSchedule-(172)-04-11-2021.xml
Inserting Record FullSchedule-(171)-04-11-2021.xml
Inserting Record FullSchedule-(170)-04-11-2021.xml
Inserting Record FullSchedule-(169)-04-11-2021.xml
Inserting Record FullSchedule-(168)-04-11-2021.xml
Inserting Record FullSchedule-(167)-04-11-2021.xml
Inserting Record FullSchedule-(166)-04-11-2021.xml
Inserting Record FullSchedule-(165)-04-11-2021.xml
Inserting Record FullSchedule-(164)-04-11-2021.xml
Inserting Record FullSchedule-(163)-04-11-2021.xml
Inserting Record FullSchedule-(162)-04-11-2021.xml
Inserting Record FullSchedule-(161)-04-11-2021.xml
Inserting Record FullSchedule-(160)-04-11-2021.xml
Inserting Record FullSchedule-(159)-04-11-2021.xml
Inserting Record FullSchedule-(0)-05-11-2021.xml
Inserting Record FullSchedule-(158)-04-11-2021.xml
Inserting Record FullSchedule-(157)-04-11-2021.xml
Inserting Record FullSchedule-(15

Inserting Record FullSchedule-(201)-03-11-2021.xml
Inserting Record FullSchedule-(8)-04-11-2021.xml
Inserting Record FullSchedule-(200)-03-11-2021.xml
Inserting Record FullSchedule-(7)-04-11-2021.xml
Inserting Record FullSchedule-(199)-03-11-2021.xml
Inserting Record FullSchedule-(6)-04-11-2021.xml
Inserting Record FullSchedule-(5)-04-11-2021.xml
Inserting Record FullSchedule-(198)-03-11-2021.xml
Inserting Record FullSchedule-(197)-03-11-2021.xml
Inserting Record FullSchedule-(196)-03-11-2021.xml
Inserting Record FullSchedule-(195)-03-11-2021.xml
Inserting Record FullSchedule-(194)-03-11-2021.xml
Inserting Record FullSchedule-(193)-03-11-2021.xml
Inserting Record FullSchedule-(4)-04-11-2021.xml
Inserting Record FullSchedule-(192)-03-11-2021.xml
Inserting Record FullSchedule-(3)-04-11-2021.xml
Inserting Record FullSchedule-(191)-03-11-2021.xml
Inserting Record FullSchedule-(190)-03-11-2021.xml
Inserting Record FullSchedule-(189)-03-11-2021.xml
Inserting Record FullSchedule-(188)-03-11-2

Inserting Record FullSchedule-(42)-03-11-2021.xml
Inserting Record FullSchedule-(41)-03-11-2021.xml
Inserting Record FullSchedule-(40)-03-11-2021.xml
Inserting Record FullSchedule-(39)-03-11-2021.xml
Inserting Record FullSchedule-(38)-03-11-2021.xml
Inserting Record FullSchedule-(37)-03-11-2021.xml
Inserting Record FullSchedule-(36)-03-11-2021.xml
Inserting Record FullSchedule-(35)-03-11-2021.xml
Inserting Record FullSchedule-(34)-03-11-2021.xml
Inserting Record FullSchedule-(33)-03-11-2021.xml
Inserting Record FullSchedule-(32)-03-11-2021.xml
Inserting Record FullSchedule-(31)-03-11-2021.xml
Inserting Record FullSchedule-(30)-03-11-2021.xml
Inserting Record FullSchedule-(29)-03-11-2021.xml
Inserting Record FullSchedule-(28)-03-11-2021.xml
Inserting Record FullSchedule-(27)-03-11-2021.xml
Inserting Record FullSchedule-(26)-03-11-2021.xml
Inserting Record FullSchedule-(25)-03-11-2021.xml
Inserting Record FullSchedule-(24)-03-11-2021.xml
Inserting Record FullSchedule-(23)-03-11-2021.xml


Inserting Record FullSchedule-(93)-02-11-2021.xml
Inserting Record FullSchedule-(92)-02-11-2021.xml
Inserting Record FullSchedule-(91)-02-11-2021.xml
Inserting Record FullSchedule-(90)-02-11-2021.xml
Inserting Record FullSchedule-(89)-02-11-2021.xml
Inserting Record FullSchedule-(88)-02-11-2021.xml
Inserting Record FullSchedule-(87)-02-11-2021.xml
Inserting Record FullSchedule-(86)-02-11-2021.xml
Inserting Record FullSchedule-(85)-02-11-2021.xml
Inserting Record FullSchedule-(84)-02-11-2021.xml
Inserting Record FullSchedule-(83)-02-11-2021.xml
Inserting Record FullSchedule-(82)-02-11-2021.xml
Inserting Record FullSchedule-(81)-02-11-2021.xml
Inserting Record FullSchedule-(80)-02-11-2021.xml
Inserting Record FullSchedule-(79)-02-11-2021.xml
Inserting Record FullSchedule-(78)-02-11-2021.xml
Inserting Record FullSchedule-(77)-02-11-2021.xml
Inserting Record FullSchedule-(76)-02-11-2021.xml
Inserting Record FullSchedule-(75)-02-11-2021.xml
Inserting Record FullSchedule-(74)-02-11-2021.xml


In [ ]:
# Part 2 - Get Data From Website


def dataparser(url, dbtime):

    rawtable = pd.read_html(url)

    df = pd.DataFrame(rawtable[0])

    df["Timestamp"] = pd.to_datetime(
        df.Timestamp, format="%m/%d/%Y %I:%M:%S %p")

    

    df['rldc'] = 'NRLDC'

    df['porting_status'] = 'N'

    finadf = df[df["Timestamp"] > dbtime]

    return finadf


finaldata = dataparser(rldc_url, maxdbtime[0])


if len(finaldata) < 1:
    print("No New  Records")
else:
    print(f'Total New Records : {len(finaldata)}')

# Part 3 - Insert Data into DB

insertquery = "INSERT INTO NTPCPowerBI.dbo.RLDC_XML_PortingStatus (FileName,FileTime,RLDC,Porting_Status) VALUES (?,?,?,'N')"
for index, row in finaldata.iterrows():
    print("Inserting Record "+str(row.Filename))
    cursor.execute(insertquery, row.Filename, row.Timestamp, row.rldc)
    print("Downloading Record "+str(row.Filename))
    # print(str(row.Filename))
    download_url = rldc_url + row.Filename

    response = requests.get(download_url)

    folderpath = r"C:\NTPC\NRLDC_XML\nrldc"
    filename = os.path.join(folderpath, row.Filename)

    # print(response.content)

    with open(filename, 'wb') as file:
        file.write(response.content)
    print("Done")
    cnxn.commit()
    # cursor.clos

for index, row in finaldata.iterrows():
    print("Downloading Record "+str(row.Filename))
    # print(str(row.Filename))
    download_url = rldc_url + row.Filename

    response = requests.get(download_url)

    folderpath = r"C:\NTPC\NRLDC_XML\nrldc"
    filename = os.path.join(folderpath, row.Filename)

    # print(response.content)

    with open(filename, 'wb') as file:
        file.write(response.content)
    print("Done")